In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
sa.__version__

'1.4.15'

In [126]:
symbols = ["AAPL", "AMZN", "C", "CAR", "GOOGL", "MSFT", "TSLA"]
start_date   = "2019-01-01"
end_date = "2019-12-31"

engine = sa.create_engine("postgresql://postgres@db/postgres")
conn = engine.connect()

metadata = sa.MetaData(engine)
metadata.reflect()
table = sa.Table("assets_1m_rth", metadata, autoload_with=engine)
cols = table.columns

query = (
    sa.sql.select(
        [
            cols.symbol,
            cols.datetime,
            cols.close,
            sa.sql.text(
                'AVG("close") OVER(PARTITION BY symbol ORDER BY datetime ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS close5'
            ),
        ]
    )
    .where(
        cols.symbol.in_(symbols),
        cols.datetime > start_date,
        cols.datetime
        < sa.sql.text(f"CAST('{end_date}' AS timestamp) + interval '1 day'"),
    )
    .order_by(cols.datetime)
)

result = conn.execute(query).fetchall()

df = pd.DataFrame.from_records(result, columns=result[0].keys())
df

,symbol,datetime,close,close5
0,AMZN,2019-01-02 09:30:00,1466.9690,1466.9690000000000000
1,C,2019-01-02 09:30:00,50.7900,50.7900000000000000
2,AAPL,2019-01-02 09:30:00,154.7800,154.7800000000000000
3,CAR,2019-01-02 09:30:00,22.1700,22.1700000000000000
4,GOOGL,2019-01-02 09:30:00,1027.9200,1027.9200000000000000
...,...,...,...,...
673725,MSFT,2019-12-31 15:59:00,157.4900,157.6150000000000000
673726,GOOGL,2019-12-31 15:59:00,1338.1200,1338.4120000000000000
673727,TSLA,2019-12-31 15:59:00,418.1700,417.9620000000000000
673728,AMZN,2019-12-31 15:59:00,1847.6400,1847.3995200000000000


In [149]:
df[(df["symbol"] == "AAPL") & (df["datetime"] > "2019-06-03") & (df["datetime"] < "2019-06-04")]

,symbol,datetime,close,close5
280756,AAPL,2019-06-03 09:30:00,176.3800,175.4380000000000000
280759,AAPL,2019-06-03 09:31:00,176.2900,175.6440000000000000
280766,AAPL,2019-06-03 09:32:00,176.7200,175.9280000000000000
280774,AAPL,2019-06-03 09:33:00,177.2400,176.3260000000000000
280776,AAPL,2019-06-03 09:34:00,177.5900,176.8440000000000000
...,...,...,...,...
283431,AAPL,2019-06-03 15:55:00,173.5200,173.1719800000000000
283441,AAPL,2019-06-03 15:56:00,173.1900,173.1499800000000000
283450,AAPL,2019-06-03 15:57:00,173.2300,173.2019800000000000
283452,AAPL,2019-06-03 15:58:00,173.2700,173.2639800000000000
